In [3]:
import torch
import lightgbm as lgb
from tqdm import tqdm
from loaders.lending_loader import load_data, mono_list
from monotonenorm import SigmaNet, GroupSort
from sklearn.metrics import balanced_accuracy_score, accuracy_score
import numpy as np
import mup
import pandas as pd

device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

In [34]:
Xtr, Ytr, Xts, Yts = load_data(get_categorical_info=False)
monotonic_constraints = [int(i in mono_list) for i in range(Xtr.shape[1])]

In [35]:
clf = lgb.LGBMRegressor(n_estimators=10000, max_depth=5, learning_rate=.1, monotone_constraint=monotonic_constraints)
clf.fit(Xtr, Ytr, early_stopping_rounds=200, eval_set=[(Xts, Yts)], eval_metric='mse', verbose=0)

LGBMRegressor(max_depth=5,
              monotone_constraint=[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                                   0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
              n_estimators=10000)

In [36]:
acc, bacc = 0,0
for i in np.linspace(0,1,100):
  acc = max(acc, accuracy_score(Yts, clf.predict(Xts)>i))
  bacc = max(bacc, balanced_accuracy_score(Yts, clf.predict(Xts)>i))
print(f"Accuracy: {acc*100:.4f}%, Balanced Accuracy: {bacc*100:.4f}%")

Accuracy: 65.9944%, Balanced Accuracy: 66.0154%


In [37]:
torch.manual_seed(13)

per_layer_lip = 2
width = 32

class Model(torch.nn.Module):
  def __init__(self, width, robust=False, sigma=False):
    super().__init__()
    if robust:
      from monotonenorm import direct_norm
      activation = lambda : GroupSort(2)
    else:
      direct_norm = lambda x, *args, **kwargs: x # make it a normal network
      activation = lambda : torch.nn.ReLU()

    self.nn = torch.nn.Sequential(
      direct_norm(torch.nn.Linear(Xtr.shape[1], width), kind="one-inf", alpha=per_layer_lip),
      activation(),
      direct_norm(torch.nn.Linear(width, width), kind="inf", alpha=per_layer_lip),
      activation(),
      direct_norm(torch.nn.Linear(width, width), kind="inf", alpha=per_layer_lip),
      activation(),
      direct_norm(torch.nn.Linear(width, 1), kind="inf", alpha=per_layer_lip),
    )
    if sigma:
      self.nn = torch.nn.Sequential(
        SigmaNet(self.nn, sigma=per_layer_lip**4, monotone_constraints=monotonic_constraints),
        torch.nn.Sigmoid()
      )
  
  def forward(self, x):
    return self.nn(x)

model = Model(width, robust=True, sigma=True)



model = model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
print('params:', sum(p.numel() for p in model.parameters()))
print(model)

params: 3073


In [39]:
Xtrt = torch.tensor(Xtr, dtype=torch.float32).to(device)
Ytrt = torch.tensor(Ytr, dtype=torch.float32).view(-1, 1).to(device)
Xtst = torch.tensor(Xts, dtype=torch.float32).to(device)
Ytst = torch.tensor(Yts, dtype=torch.float32).view(-1, 1).to(device)

#between -1 and 1
# min_ = Xtrt.min()
# max_ = Xtrt.max()
# Xtrt = (Xtrt - min_) / (max_ - min_) * 2 - 1 
# Xtst = (Xtst - min_) / (max_ - min_) * 2 - 1

#std = 1
mean = Xtrt.mean(0)
std = Xtrt.std(0)
Xtrt = (Xtrt - mean) / std
Xtst = (Xtst - mean) / std


dataloader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(Xtrt, Ytrt), batch_size=int(2**9), shuffle=True)
max_acc = 0

bar = tqdm(range(100))
for i in bar:
  for Xi, yi in dataloader:
    y_pred = model(Xi)
    losstr = torch.nn.functional.binary_cross_entropy(y_pred, yi)
    optimizer.zero_grad()
    losstr.backward()
    optimizer.step()
    #scheduler.step()

  with torch.no_grad():
    y_predts = model(Xtst)
    lossts = torch.nn.functional.binary_cross_entropy(y_predts, Ytst)
    if i % 1 == 0:
      acc = 0
      for i in np.linspace(0, 1, 50):
        acc = max(acc, accuracy_score(Ytst.cpu().numpy(), y_predts.cpu().numpy()>i))

    max_acc = max(max_acc, acc)
    bar.set_description(f'Loss: {losstr.item():.4f} {lossts.item():.4f}, acc: {acc.item():.4f}, max_acc: {max_acc:.4f}')

Loss: 0.6306 0.6356, acc: 0.6525, max_acc: 0.6528:   4%|▍         | 44/1000 [02:51<1:04:05,  4.02s/it]